In [ ]:
%load_ext autoreload
%autoreload 2

In [20]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [48]:
df = pd.read_csv('../../data/clean_data/train_df_preprocessed.csv')
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,No Alley,Reg,Lvl,AllPub,...,0,No Pool,No Fence,No feature,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,No Alley,Reg,Lvl,AllPub,...,0,No Pool,No Fence,No feature,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,No Alley,IR1,Lvl,AllPub,...,0,No Pool,No Fence,No feature,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,No Alley,IR1,Lvl,AllPub,...,0,No Pool,No Fence,No feature,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,No Alley,IR1,Lvl,AllPub,...,0,No Pool,No Fence,No feature,0,12,2008,WD,Normal,250000


*New Features Creation*

* Age/Date Features

In [49]:
df['HouseAge'] = df['YrSold'] - df['YearBuilt']
df['YearsSinceRemodel'] = df['YrSold'] - df['YearRemodAdd']
df['WasRemodeled'] = df.apply(lambda row: 1 if row['YearRemodAdd'] > row['YearBuilt'] else 0, axis=1)
df['GarageAge'] = df['YrSold'] - df['GarageYrBlt']
df['HouseAgeCategory'] = pd.cut(df['HouseAge'], bins=[0, 10, 20, 50, 100, 200], labels=['New', 'Recent', 'Modern', 'Old', 'Very Old'])
df['SeasonSold'] = df['MoSold'].apply(lambda x: 'Spring' if x in [3, 4, 5] else 'Summer' if x in [6, 7, 8] else 'Fall' if x in [9, 10, 11] else 'Winter')

* Quality and Condition Features

In [50]:
df['QualCondMult'] = df['OverallQual'] * df['OverallCond']
df['QualCondRatio'] = df['OverallQual'] / df['OverallCond']

In [51]:
# Create new numerical features for quality and condition
qc_mapping = {
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 1
}

qc_features = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'GarageQual', 'GarageCond', 'PoolQC', 'FireplaceQu']
for feature in qc_features:
    df[feature] = df[feature].map(qc_mapping).fillna(0)

In [52]:
df['OverallQualScore'] = (
    df['ExterQual'] + 
    df['ExterCond'] +
    df['BsmtQual'] +
    df['BsmtCond'] +
    df['HeatingQC'] +
    df['KitchenQual'] +
    df['GarageQual'] +
    df['GarageCond'] +
    df['PoolQC'] +
    df['FireplaceQu']
)

* Has {...} Features

In [58]:
df['HasAlley'] = df['Alley'].apply(lambda x: 0 if x == 'No Alley' else 1)
df['HasMasVnr'] = df['MasVnrType'].apply(lambda x: 0 if x == 'No masonry veneer' else 1)
df['HasBsmt'] = df['BsmtQual'].apply(lambda x: 0 if x == 0 else 1)
df['HasFireplace'] = df['FireplaceQu'].apply(lambda x: 0 if x == 0 else 1)
df['HasGarage'] = df['GarageQual'].apply(lambda x: 0 if x == 0 else 1)
df['HasPool'] = df['PoolQC'].apply(lambda x: 0 if x == 0 else 1)
df['HasFence'] =df['Fence'].apply(lambda x: 0 if x == 'No Fence' else 1)
df['HasPorch'] = df[['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']].sum(axis=1).apply(lambda x: 1 if x > 0 else 0)

* Footage/Area Features

In [59]:
df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
df['TotalPorchSF'] = df['OpenPorchSF'] + df['EnclosedPorch'] + df['3SsnPorch'] + df['ScreenPorch']
df['TotalBsmtFinSF'] = df['BsmtFinSF1'] + df['BsmtFinSF2']
df['HouseToLotRatio'] = df['GrLivArea'] / df['LotArea']

* Neighbourhood Related Features

In [60]:
rich_neighborhoods = df.groupby('Neighborhood')['SalePrice'].mean().nlargest(10).index
df['IsRichNeighborhood'] = df['Neighborhood'].apply(lambda x: 1 if x in rich_neighborhoods else 0)

*Feature transformation*

* Categorical Features

In [14]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,HasFireplace,HasGarage,HasPool,HasFence,HasPorch,TotalSF,TotalPorchSF,TotalBsmtFinSF,HouseToLotRatio,IsRichNeighborhood
0,1,60,RL,65.0,8450,Pave,No Alley,Reg,Lvl,AllPub,...,0,1,0,0,1,2566,61,706,0.202367,1
1,2,20,RL,80.0,9600,Pave,No Alley,Reg,Lvl,AllPub,...,1,1,0,0,0,2524,0,978,0.131458,1
2,3,60,RL,68.0,11250,Pave,No Alley,IR1,Lvl,AllPub,...,1,1,0,0,1,2706,42,486,0.158756,1
3,4,70,RL,60.0,9550,Pave,No Alley,IR1,Lvl,AllPub,...,1,1,0,0,1,2473,307,216,0.179791,1
4,5,60,RL,84.0,14260,Pave,No Alley,IR1,Lvl,AllPub,...,1,1,0,0,1,3343,84,655,0.154137,1


In [15]:
df['MSSubClass'] = df['MSSubClass'].astype(object)

In [25]:
def categorical_features_ohe(df: pd.DataFrame, column: pd.DataFrame.columns) -> pd.DataFrame:
    ohe = OneHotEncoder(sparse_output=False)
    
    ohe.fit(df[[column]])
    ohe_columns = ohe.transform(df[[column]])
    ohe_df = pd.DataFrame(ohe_columns, columns=ohe.get_feature_names_out([column]))
    
    df = pd.concat([df.reset_index(drop=True), ohe_df.reset_index(drop=True)], axis=1)
    df = df.drop(columns=[column], axis=1)
    return df

In [26]:
categorical_features = df.select_dtypes(include=['object']).columns

for feature in categorical_features:
    df = categorical_features_ohe(df, feature)
    
df.head()

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SeasonSold_Fall,SeasonSold_Spring,SeasonSold_Summer,SeasonSold_Winter
0,1,65.0,8450,7,5,2003,2003,196.0,4,3,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,2,80.0,9600,6,8,1976,1976,0.0,3,3,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,3,68.0,11250,7,5,2001,2002,162.0,4,3,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,4,60.0,9550,7,5,1915,1970,0.0,3,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,84.0,14260,8,5,2000,2000,350.0,4,3,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
# TODO apply np.log1p for right-skewed features, see outliers section in data_processing.ipynb